In [1]:
#Libraries
import pandas as pd
import psycopg2
import folium

C:\Users\deyvid.linoba\AppData\Local\Temp\ipykernel_19084\2267923054.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#functions

#connection Postgresql
def con_postgres(url,user,password,port):
  
  conn = psycopg2.connect(host=url,
                                user=user,
                                password=password,
                                port=port)
  return conn

def create_default_table_school(conn):
  mesage = 'default table already exists'
  cursor = conn.cursor()
  cursor.execute("SELECT EXISTS (SELECT relname FROM pg_class WHERE relname = 'school')")

  if not cursor.fetchone()[0]:
      cursor.execute('''CREATE TABLE public.school (
                                                  id bigserial NOT NULL,
                                                  name varchar(255),
                                                  latitude float,
                                                  longitude float,
                                                  location geography(Point, 4326) NOT NULL
                                                  );
                    CREATE INDEX idx_geom_table_school ON school USING GIST(location)''')
      conn.commit()
      mesage = 'default table created'
  return mesage

def read_csv(path):
  #choosing specific columns
  columns = ['UNIDADE_ ESCOLAR','LATITUDE','LONGITUDE']

  df = pd.read_csv(path, sep=';', encoding='ANSI', usecols=columns)

  #removing lines NAN
  df=df.dropna(axis=0)

  df['UNIDADE_ ESCOLAR'] = df['UNIDADE_ ESCOLAR'].str.title()
  df['LATITUDE'] = df['LATITUDE'].str.replace(',','.').astype(float) 
  df['LONGITUDE'] = df['LONGITUDE'].str.replace(',','.').astype(float)

  #remove invalid coordinates
  invalid_coordinates = df[(df['LONGITUDE'] < -180) | (df['LONGITUDE'] > 180) | (df['LATITUDE'] < -90) | (df['LATITUDE'] > 90)].index

  df.drop(invalid_coordinates , inplace=True)

  #droping index
  df.reset_index(drop=True,inplace=True)
  return df

def save_df_postgres(conn,data):
   mesage = 'Data inserted'
   cursor = conn.cursor()
   for i, d in data.iterrows():
    try:
      cursor.execute('INSERT INTO public.school (name,latitude,longitude, location) VALUES (%s,%s,%s,ST_SetSRID(ST_MakePoint(%s,%s), 4326));',(d['UNIDADE_ ESCOLAR'],d['LATITUDE'],d['LONGITUDE'],d['LATITUDE'],d['LONGITUDE']))
      conn.commit()
    except Exception as error:
      # handle the exception
      mesage = f"An exception occurred: {error}"
   return mesage

def search_school_nearby(lat,long,maxdistance, conn):
  cursor = conn.cursor()
  cursor.execute("""SELECT s.name,s.latitude,s.longitude,st_distancesphere(s.location::geometry,ST_SetSRID(ST_MakePoint(%s,%s), 4326))
                  FROM school s
                  WHERE ST_DWithin(s.location, ST_SetSRID(ST_MakePoint(%s,%s), 4326)::geography, %s)
                  ORDER BY 4;""",(lat,long,lat,long,maxdistance))
  result = cursor.fetchall()
  conn.close()
  return result

def plot_map(lat,long,result):
  map=folium.Map(location=[lat,long],zoom_start=13)
  map.add_child(folium.Marker(location=[lat,long],popup='You Are Here',icon=folium.Icon(color='blue')))
  
  if len(result) > 0:
    for x,obj in enumerate(result):
      if x == 0:
         map.add_child(folium.Marker(location=[obj[1], obj[2]],popup=f'Closer {obj[0]}',icon=folium.Icon(color='green')))
      else:
         map.add_child(folium.Marker(location=[obj[1], obj[2]],icon=folium.Icon(color='red')))
  return map

In [3]:
#CREATE CONNECTION
conn = con_postgres("127.0.0.1","postgres","postgres","5432")
#CREATE DEFAULT TABLE
create_default_table_school(conn)

#read CSV
path = '4.4.3-unidades-educacionais-no-municipio_31.12.2023.csv'
df = read_csv(path)

#save dataframe to Postgresql database
result = save_df_postgres(conn,df)

#setting your coordinate
lat = -3.726027722459586
long = -38.53996724673096
maxdistance = 1000

#querying database
result = search_school_nearby(lat,long,maxdistance, conn)

#ploting map
map = plot_map(lat,long,result)

map